# Data loading

In [1]:
import numpy as np
from matplotlib import pyplot as plt

import torch
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

import pickle

In [2]:
data_name = '20071116_100days_15lb_0.7Distance_Falseloops'

dataset = torch.load(f'/finance_data/{data_name}')

In [3]:
def data_info(dataset):
    print(f'Number of samples = {dataset.snapshot_count}, per sample:')
    print(f'Number of vertices: {dataset[0].x.shape[0]}')
    print(f'Number of time steps: {dataset[0].x.shape[1]}')
    print(f'Number of edge types: {dataset[0].edge_type.max().item() + 1}')

In [4]:
data_info(dataset)

Number of samples = 100, per sample:
Number of vertices: 200
Number of time steps: 16
Number of edge types: 1


# Model

In [5]:
class GDN(nn.Module):
    def __init__(
        self,
        num_steps: int,
        min_beta: float,
        max_beta: float,
        num_nodes: int,
        feature_size: int,
    ):
        super(GDN, self).__init__()
        self.num_steps = num_steps
        self.betas = torch.linspace(min_beta, max_beta, num_steps)
        self.alphas = 1 - self.betas
        self.alpha_bars = torch.tensor([torch.prod(self.alphas[:i + 1]) for i in range(len(self.alphas))])
        
        self._GCN = GCNConv(feature_size, feature_size)  # By default this adds self loops and normalises
        
        self._PIUNet = PIUNet(num_nodes, num_steps)
        
        self._DDPM = DDPM(self._PIUNet, num_steps, self.betas, self.alphas, self.alpha_bars)
        
        self._DDRM = DDRM(self._DDPM, num_nodes, feature_size)
        
        self._linear = nn.Linear(feature_size, 1)
        
        
    def forward(
        self,
        x: torch.Tensor,  # [200, 16]
        edge_index: torch.Tensor,
        edge_type: torch.Tensor,
        edge_attr: torch.Tensor = None,
        return_sample: bool = False,
    ) -> torch.Tensor:

        h = self._GCN(x,  edge_index)  # [200, 16]
        
        if return_sample==False:        
            h = self._linear(h)  # [200, 1]
            h = torch.squeeze(h)  # [200]
        
        return h


# The PIUNet code is adapted from Pulfer's DDPM code: https://github.com/BrianPulfer/PapersReimplementations/tree/main/ddpm
class PIUNet(nn.Module):
    def __init__(
        self,
        h: int,  # Height of input matrix, for us h=200
        num_steps: int,
        time_emb_dim: int = 100
    ):
        super(PIUNet, self).__init__()

        # Sinusoidal embedding
        self.time_embed = nn.Embedding(num_steps, time_emb_dim)
        self.time_embed.weight.data = sinusoidal_embedding(num_steps, time_emb_dim)
        self.time_embed.requires_grad_(False)

        # Downsampling regime
        self.te1 = self._make_te(time_emb_dim, 1)
        self.b1 = nn.Sequential(
            PICNN((1, h, 16), 1, 10),
            PICNN((10, h, 16), 10, 10),
            PICNN((10, h, 16), 10, 10)
        )
        self.down1 = nn.Conv2d(10, 10, (1,4), (1,2), (0,1))

        self.te2 = self._make_te(time_emb_dim, 10)
        self.b2 = nn.Sequential(
            PICNN((10, h, 8), 10, 20),
            PICNN((20, h, 8), 20, 20),
            PICNN((20, h, 8), 20, 20)
        )
        self.down2 = nn.Conv2d(20, 20, (1,4), (1,2), (0,1))

        self.te3 = self._make_te(time_emb_dim, 20)
        self.b3 = nn.Sequential(
            PICNN((20, h, 4), 20, 40),
            PICNN((40, h, 4), 40, 40),
            PICNN((40, h, 4), 40, 40)
        )
        self.down3 = nn.Sequential(
            nn.Conv2d(40, 40, (1,2), 1),
            nn.SiLU(),
            nn.Conv2d(40, 40, (1,3), 1, (0,1))
        )

        # Bottleneck point
        self.te_mid = self._make_te(time_emb_dim, 40)
        self.b_mid = nn.Sequential(
            PICNN((40, h, 3), 40, 20),
            PICNN((20, h, 3), 20, 20),
            PICNN((20, h, 3), 20, 40)
        )

        # Up sampling regime
        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(40, 40, (1,3), 1, (0,1)),
            nn.SiLU(),
            nn.ConvTranspose2d(40, 40, (1,2), 1)
        )

        self.te4 = self._make_te(time_emb_dim, 80)
        self.b4 = nn.Sequential(
            PICNN((80, h, 4), 80, 40),
            PICNN((40, h, 4), 40, 20),
            PICNN((20, h, 4), 20, 20)
        )

        self.up2 = nn.ConvTranspose2d(20, 20, (1,4), (1,2), (0,1))
        self.te5 = self._make_te(time_emb_dim, 40)
        self.b5 = nn.Sequential(
            PICNN((40, h, 8), 40, 20),
            PICNN((20, h, 8), 20, 10),
            PICNN((10, h, 8), 10, 10)
        )

        self.up3 = nn.ConvTranspose2d(10, 10, (1,4), (1,2), (0,1))
        self.te_out = self._make_te(time_emb_dim, 20)
        self.b_out = nn.Sequential(
            PICNN((20, h, 16), 20, 10),
            PICNN((10, h, 16), 10, 10),
            PICNN((10, h, 16), 10, 10, normalize=False)
        )

        self.conv_out = nn.Conv2d(10, 1, (1,3), 1, (0,1))

    def forward(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
    ) -> torch.Tensor:
    
        # x is dim [N, 1, 200, 16], where N = batch size
        t = self.time_embed(t)
        n = len(x)
        out1 = self.b1(x + self.te1(t).reshape(n, -1, 1, 1))  # [N, 10, 200, 16]
        out2 = self.b2(self.down1(out1) + self.te2(t).reshape(n, -1, 1, 1))  # [N, 20, 200, 8]
        out3 = self.b3(self.down2(out2) + self.te3(t).reshape(n, -1, 1, 1))  # [N, 40, 200, 4]
        out_mid = self.b_mid(self.down3(out3) + self.te_mid(t).reshape(n, -1, 1, 1))  # [N, 40, 200, 3]
        out4 = torch.cat((out3, self.up1(out_mid)), dim=1)  # [N, 80, 200, 4]        
        out4 = self.b4(out4 + self.te4(t).reshape(n, -1, 1, 1))  # [N, 20, 200, 4]
        out5 = torch.cat((out2, self.up2(out4)), dim=1)  # [N, 40, 200, 8]
        out5 = self.b5(out5 + self.te5(t).reshape(n, -1, 1, 1))  # [N, 10, 200, 8]
        out = torch.cat((out1, self.up3(out5)), dim=1)  # [N, 20, 200, 16]        
        out = self.b_out(out + self.te_out(t).reshape(n, -1, 1, 1))  # [N, 10, 200, 16]
        out = self.conv_out(out)  # [N, 1, 200, 16]

        return out
    
    # The single hidden layer MLP which is used to map positional embeddings
    def _make_te(self, dim_in, dim_out):
        return nn.Sequential(
        nn.Linear(dim_in, dim_out),
        nn.SiLU(),
        nn.Linear(dim_out, dim_out)
      )

# The "PICNN" called on is the below 1-D CNN class:
class PICNN(nn.Module):
    def __init__(
        self,
        shape: tuple,
        in_c: int,
        out_c: int,
        kernel_size: tuple = (1,3),
        stride: int = 1,
        padding: tuple = (0,1),
        normalize: bool = True,
    ):
        super(PICNN, self).__init__()
        self.ln = nn.LayerNorm(shape)
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size, stride, padding)
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size, stride, padding)
        self.activation = nn.SiLU()
        self.normalize = normalize

    def forward(
        self,
        x: torch.Tensor,
    ) -> torch.Tensor:
    
        out = self.ln(x) if self.normalize else x
        out = self.conv1(out)
        out = self.activation(out)
        out = self.conv2(out)
        out = self.activation(out)
        
        return out

# The "sinusoidal_embedding" called on is the below function, this
# returns the standard positional embedding
def sinusoidal_embedding(n, d):
    embedding = torch.zeros(n, d)
    wk = torch.tensor([1 / 10_000 ** (2 * j / d) for j in range(d)])
    wk = wk.reshape((1, d))
    t = torch.arange(n).reshape((n, 1))
    embedding[:,::2] = torch.sin(t * wk[:,::2])
    embedding[:,1::2] = torch.cos(t * wk[:,::2])

    return embedding



class DDPM(nn.Module):
    def __init__(
        self,
        network,
        num_steps,
        betas,
        alphas,
        alpha_bars,
    ):
        super(DDPM, self).__init__()
        self.num_steps = num_steps
        self.network = network  # Will pass in PIU-Net as network
        self.betas = betas
        self.alphas = alphas
        self.alpha_bars = alpha_bars

    def forward(
        self,
        x0: torch.Tensor,
        t: torch.Tensor,
        epsilon: torch.Tensor = None,
    ) -> torch.Tensor:
    
        n, c, h, w = x0.shape  # n = batch size, c = num channels, h = height, w = width
        a_bar = self.alpha_bars[t]

        if epsilon is None:
            epsilon = torch.randn(n, c, h, w)
        
        noisy = a_bar.sqrt().reshape(n, 1, 1, 1)*x0 + (1 - a_bar).sqrt().reshape(n, 1, 1, 1)*epsilon
        return noisy

    def backward(self, x, t):
        # Runs each image in the batch through the network, each for their
        # random time step t, the network returns its estimation of
        # the noise that was added in the last step
        return self.network(x, t)



class DDRM(nn.Module):
    def __init__(
        self,
        ddpm,
        h: int,
        w: int,
    ):
        super(DDRM, self).__init__()
        self.ddpm = ddpm
        self.h = h
        self.w = w
        
    def forward(
        self,
        y: torch.Tensor,
        sigma_y: float,
    ) -> torch.Tensor:
        
        ddpm = self.ddpm
        h = self.h
        w = self.w
        
        for idx, t in enumerate(list(range(ddpm.num_steps))[::-1]):
            # Using \sigma_t^2 = \beta_t, Ho et al. found the model to be robust to the \sigma_t choice
            beta_t = ddpm.betas[t]
            sigma_t = beta_t.sqrt()

            # To satisfy Theorem 1
            eta = 1
            eta_b = (2*sigma_t**2)/(sigma_t**2 + sigma_y**2)
            
            z_t = torch.randn(h, w) 

            if idx==0:
                assert sigma_y<=sigma_t, f'Error: sigma_y = {sigma_y} > {sigma_t} = sigma_T, decrease sigma_y value.'
                x_t = y + (sigma_t**2 - sigma_y**2).sqrt() * z_t


            else:           
                time_tensor = (torch.ones(1, 1) * (t+1)).long()
                tilde_epsilon_t = ddpm.backward(x_t[None, None, :], time_tensor)[0][0]  # here x_t corresponds to t+1

                alpha_t = ddpm.alphas[t]
                alpha_t_bar = ddpm.alpha_bars[t]
                
                tilde_x_t = (1/alpha_t.sqrt()) * (x_t - (beta_t/((1 - alpha_t_bar).sqrt())) * tilde_epsilon_t)
                
                if sigma_t<sigma_y:
                    x_t = tilde_x_t + sigma_t * z_t
                else:
                    x_t = (1-eta_b)*tilde_x_t + eta_b*y + (sigma_t**2 - (sigma_y**2)*(eta_b**2)).sqrt() * z_t

        return x_t

# Rolling train/test algorithms

In [6]:
def custom_loss(y_hat, y, gamma, delta):
    loss = delta*torch.mean(-torch.tanh(gamma*y_hat)*y) + (1-delta)*torch.mean((y_hat-y)**2)
    return loss

def pnl_function(y_hat, y):
    pnl = torch.mean(torch.sign(y_hat)*y)
    return pnl

In [7]:
def rolling_train_test(model, data, GDN_epochs, DDPM_epochs, gamma, delta, GDN_lr, DDPM_lr, DDPM_lb, training_lb, GCN):    
    train_losses, train_pnls, test_losses, test_pnls, test_ys, all_batches = [], [], [], [], [], []
    T = data.snapshot_count
    
    for t in range(training_lb, T):
        print(f' ----- On train/test for day {t} ----- ')
        
        if GCN==True:
            b = T
        else:
            b = DDPM_lb+training_lb
        
        if t<b:
            model_passing_outputs = model_passing(model, data, GDN_epochs, gamma, delta, GDN_lr, training_lb,
                                                  t, False, GCN)
            all_batches.append(model_passing_outputs[0])
            train_losses.append(model_passing_outputs[1])
            train_pnls.append(model_passing_outputs[2])
            test_losses.append(model_passing_outputs[3])
            test_pnls.append(model_passing_outputs[4])
            test_ys.append(model_passing_outputs[5])
            print(f'Current test pnl = {model_passing_outputs[4]}')
            
        else:
            model.train()
            previous_batches = np.array(all_batches[-DDPM_lb:])
            previous_batches = torch.tensor(previous_batches)
            
            DDPMoptim = Adam(model._DDPM.parameters(), lr=DDPM_lr)
            mse = nn.MSELoss()
            num_steps = model.num_steps
            
            for epoch in range(DDPM_epochs):
                epoch_loss = 0.0
                
                for batch in previous_batches:
                    x0 = batch                    
                    n = len(x0)
                    epsilon = torch.randn_like(x0)
                    time = torch.randint(0, num_steps, (n,))
                    noisy_imgs = model._DDPM(x0, time, epsilon)
                    epsilon_theta = model._DDPM.backward(noisy_imgs, time.reshape(n, -1))
                    DDPM_loss = mse(epsilon_theta, epsilon)
                    
                    DDPMoptim.zero_grad()
                    DDPM_loss.backward()
                    DDPMoptim.step()
                    
                    epoch_loss += DDPM_loss.item()

                print(f"Loss at DDPM epoch {epoch + 1}: {epoch_loss/DDPM_lb:.6f}")
            
            model_passing_outputs = model_passing(model, data, GDN_epochs, gamma, delta, GDN_lr, training_lb,
                                                  t, True, GCN)
            all_batches.append(model_passing_outputs[0])
            train_losses.append(model_passing_outputs[1])
            train_pnls.append(model_passing_outputs[2])
            test_losses.append(model_passing_outputs[3])
            test_pnls.append(model_passing_outputs[4])
            test_ys.append(model_passing_outputs[5])
            print(f'Current test pnl = {model_passing_outputs[4]:.6f}')
            
    return train_losses, train_pnls, test_losses, test_pnls, np.array(test_ys)

In [8]:
def model_passing(model, data, GDN_epochs, gamma, delta, GDN_lr, training_lb, t, use_DDRM, GCN):    
    current_batch, train_t_losses, train_t_pnls = [], [], []
    
    params = list(model._GCN.parameters()) + list(model._linear.parameters())
    optim = Adam(params, lr=GDN_lr)
    
    model.eval()
    x_data = []
    for i in range(t-training_lb, t+1):
        m = data[i].x
        
        if use_DDRM==True:
            print(f'Sampling for day {i-t+training_lb+1}/{training_lb+1} which is day {i}')
            std_dev = m.std()
            
            # Below we standardise before denoising the sample via the DDRM and then scale back
            # up post-DDRM
            m = m/std_dev            
            x_data.append((model._DDRM(m, 0.05).detach().numpy())*std_dev.item())
        else:
            x_data.append(m.detach().numpy())
    
    x_data = np.array(x_data)
    x_data = torch.tensor(x_data)
    
    
    model.train()
    for epoch in range(GDN_epochs):
        train_epoch_loss = 0
        train_pnl_loss = 0
        
        for i, t_ in enumerate(range(t-training_lb, t)):
            snapshot = data[t_]
            y_hat = model(x_data[i], snapshot.edge_index, snapshot.edge_type, snapshot.edge_attr, False)
            train_loss = custom_loss(y_hat, snapshot.y, gamma, delta)
            train_pnl_loss += pnl_function(y_hat, snapshot.y).item()
                
            optim.zero_grad()
            train_loss.backward()
            optim.step()

            train_epoch_loss += train_loss.item()
        
        print(f'On epoch {epoch+1} of day {t} training, loss = {train_epoch_loss/training_lb:.8f}')

        # We divide by training_lb so can compare fairly for varying training_lb size
        train_t_losses.append(train_epoch_loss/training_lb)
        train_t_pnls.append(train_pnl_loss/training_lb)
    
    
    model.eval()
    if GCN==False:
        for i, t_ in enumerate(range(t-training_lb, t+1)):
            snapshot = data[t_]
            x_0_hat = model(x_data[i], snapshot.edge_index, snapshot.edge_type, snapshot.edge_attr, True)
            x_0_hat = x_0_hat/x_0_hat.std()  # So all of DDPM training samples are standardised
            x_0_hat = x_0_hat[None, :]
            current_batch.append(x_0_hat.detach().numpy())
    
    snapshot = data[t]
    y_hat_t = model(x_data[-1], snapshot.edge_index, snapshot.edge_type, snapshot.edge_attr, False)
    test_t_loss = custom_loss(y_hat_t, snapshot.y, gamma, delta).item()
    test_t_pnl = pnl_function(y_hat_t, snapshot.y).item()
    
    return current_batch, train_t_losses, train_t_pnls, test_t_loss, test_t_pnl, (y_hat_t.detach().numpy(), snapshot.y.detach().numpy())

In [9]:
data = dataset
data_info(data)

Number of samples = 100, per sample:
Number of vertices: 200
Number of time steps: 16
Number of edge types: 1


# Training/testing

In [10]:
num_steps = 500
min_beta = 10**-4
max_beta = 0.02
num_nodes = 200
feature_size = 16

gdn = GDN(num_steps, min_beta, max_beta, num_nodes, feature_size)

In [11]:
gdn

GDN(
  (_GCN): GCNConv(16, 16)
  (_PIUNet): PIUNet(
    (time_embed): Embedding(500, 100)
    (te1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): SiLU()
      (2): Linear(in_features=1, out_features=1, bias=True)
    )
    (b1): Sequential(
      (0): PICNN(
        (ln): LayerNorm((1, 200, 16), eps=1e-05, elementwise_affine=True)
        (conv1): Conv2d(1, 10, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (conv2): Conv2d(10, 10, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (activation): SiLU()
      )
      (1): PICNN(
        (ln): LayerNorm((10, 200, 16), eps=1e-05, elementwise_affine=True)
        (conv1): Conv2d(10, 10, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (conv2): Conv2d(10, 10, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (activation): SiLU()
      )
      (2): PICNN(
        (ln): LayerNorm((10, 200, 16), eps=1e-05, elementwise_affine=True)
        (conv1): Conv2d(10, 10, kern

In [12]:
GDN_epochs = 25
DDPM_epochs = 5
gamma = 0.75
delta = 0.75
GDN_lr = 0.001
DDPM_lr = 0.001
DDPM_lb = 10
training_lb = 5
GCN = True  # 'GCN = True' = GCN only (baseline comparison), 'GCN = False' = full GDN

print(f'Will be training/testing on {data.snapshot_count - training_lb} days')
if GCN==False:
    model_type = 'GDN'
    print('In GDN mode:')
    d = data.snapshot_count - training_lb - DDPM_lb
    if d>0:  
        print(f'The last {d} days will use DDPM/DDRM')
    else:
        print('No days will use DDPM/DDRM')
else:
    model_type = 'GCN'
    print('In GCN mode')
print('')

outputs_GDN = rolling_train_test(gdn, data, GDN_epochs, DDPM_epochs, gamma, delta, GDN_lr, DDPM_lr, DDPM_lb, 
                             training_lb, GCN)

Will be training/testing on 95 days
In GCN mode

 ----- On train/test for day 5 ----- 
On epoch 1 of day 5 training, loss = 0.00551004
On epoch 2 of day 5 training, loss = 0.00453584
On epoch 3 of day 5 training, loss = 0.00363193
On epoch 4 of day 5 training, loss = 0.00280288
On epoch 5 of day 5 training, loss = 0.00206414
On epoch 6 of day 5 training, loss = 0.00143306
On epoch 7 of day 5 training, loss = 0.00092467
On epoch 8 of day 5 training, loss = 0.00054730
On epoch 9 of day 5 training, loss = 0.00029820
On epoch 10 of day 5 training, loss = 0.00016061
On epoch 11 of day 5 training, loss = 0.00010480
On epoch 12 of day 5 training, loss = 0.00009504
On epoch 13 of day 5 training, loss = 0.00010057
On epoch 14 of day 5 training, loss = 0.00010399
On epoch 15 of day 5 training, loss = 0.00010131
On epoch 16 of day 5 training, loss = 0.00009564
On epoch 17 of day 5 training, loss = 0.00009070
On epoch 18 of day 5 training, loss = 0.00008800
On epoch 19 of day 5 training, loss = 0.

On epoch 14 of day 11 training, loss = 0.00005446
On epoch 15 of day 11 training, loss = 0.00005384
On epoch 16 of day 11 training, loss = 0.00005329
On epoch 17 of day 11 training, loss = 0.00005269
On epoch 18 of day 11 training, loss = 0.00005209
On epoch 19 of day 11 training, loss = 0.00005154
On epoch 20 of day 11 training, loss = 0.00005104
On epoch 21 of day 11 training, loss = 0.00005054
On epoch 22 of day 11 training, loss = 0.00005005
On epoch 23 of day 11 training, loss = 0.00004959
On epoch 24 of day 11 training, loss = 0.00004915
On epoch 25 of day 11 training, loss = 0.00004873
Current test pnl = 0.0011091406922787428
 ----- On train/test for day 12 ----- 
On epoch 1 of day 12 training, loss = 0.00006937
On epoch 2 of day 12 training, loss = 0.00006564
On epoch 3 of day 12 training, loss = 0.00006598
On epoch 4 of day 12 training, loss = 0.00006390
On epoch 5 of day 12 training, loss = 0.00006201
On epoch 6 of day 12 training, loss = 0.00006256
On epoch 7 of day 12 train

On epoch 25 of day 17 training, loss = 0.00004830
Current test pnl = -0.004036452621221542
 ----- On train/test for day 18 ----- 
On epoch 1 of day 18 training, loss = 0.00007780
On epoch 2 of day 18 training, loss = 0.00007426
On epoch 3 of day 18 training, loss = 0.00007348
On epoch 4 of day 18 training, loss = 0.00007153
On epoch 5 of day 18 training, loss = 0.00006976
On epoch 6 of day 18 training, loss = 0.00006852
On epoch 7 of day 18 training, loss = 0.00006749
On epoch 8 of day 18 training, loss = 0.00006625
On epoch 9 of day 18 training, loss = 0.00006496
On epoch 10 of day 18 training, loss = 0.00006380
On epoch 11 of day 18 training, loss = 0.00006277
On epoch 12 of day 18 training, loss = 0.00006176
On epoch 13 of day 18 training, loss = 0.00006072
On epoch 14 of day 18 training, loss = 0.00005972
On epoch 15 of day 18 training, loss = 0.00005878
On epoch 16 of day 18 training, loss = 0.00005788
On epoch 17 of day 18 training, loss = 0.00005700
On epoch 18 of day 18 trainin

On epoch 9 of day 24 training, loss = 0.00007748
On epoch 10 of day 24 training, loss = 0.00007745
On epoch 11 of day 24 training, loss = 0.00007722
On epoch 12 of day 24 training, loss = 0.00007692
On epoch 13 of day 24 training, loss = 0.00007680
On epoch 14 of day 24 training, loss = 0.00007663
On epoch 15 of day 24 training, loss = 0.00007640
On epoch 16 of day 24 training, loss = 0.00007626
On epoch 17 of day 24 training, loss = 0.00007612
On epoch 18 of day 24 training, loss = 0.00007594
On epoch 19 of day 24 training, loss = 0.00007580
On epoch 20 of day 24 training, loss = 0.00007566
On epoch 21 of day 24 training, loss = 0.00007551
On epoch 22 of day 24 training, loss = 0.00007539
On epoch 23 of day 24 training, loss = 0.00007526
On epoch 24 of day 24 training, loss = 0.00007513
On epoch 25 of day 24 training, loss = 0.00007502
Current test pnl = -0.0027250745333731174
 ----- On train/test for day 25 ----- 
On epoch 1 of day 25 training, loss = 0.00008021
On epoch 2 of day 25 

On epoch 4 of day 31 training, loss = 0.00003377
On epoch 5 of day 31 training, loss = 0.00003087
On epoch 6 of day 31 training, loss = 0.00002709
On epoch 7 of day 31 training, loss = 0.00002630
On epoch 8 of day 31 training, loss = 0.00002596
On epoch 9 of day 31 training, loss = 0.00002454
On epoch 10 of day 31 training, loss = 0.00002274
On epoch 11 of day 31 training, loss = 0.00002126
On epoch 12 of day 31 training, loss = 0.00002046
On epoch 13 of day 31 training, loss = 0.00001965
On epoch 14 of day 31 training, loss = 0.00001857
On epoch 15 of day 31 training, loss = 0.00001745
On epoch 16 of day 31 training, loss = 0.00001655
On epoch 17 of day 31 training, loss = 0.00001581
On epoch 18 of day 31 training, loss = 0.00001504
On epoch 19 of day 31 training, loss = 0.00001423
On epoch 20 of day 31 training, loss = 0.00001347
On epoch 21 of day 31 training, loss = 0.00001280
On epoch 22 of day 31 training, loss = 0.00001218
On epoch 23 of day 31 training, loss = 0.00001156
On epo

...

On epoch 16 of day 69 training, loss = 0.00006103
On epoch 17 of day 69 training, loss = 0.00006038
On epoch 18 of day 69 training, loss = 0.00005982
On epoch 19 of day 69 training, loss = 0.00005936
On epoch 20 of day 69 training, loss = 0.00005893
On epoch 21 of day 69 training, loss = 0.00005847
On epoch 22 of day 69 training, loss = 0.00005799
On epoch 23 of day 69 training, loss = 0.00005753
On epoch 24 of day 69 training, loss = 0.00005711
On epoch 25 of day 69 training, loss = 0.00005671
Current test pnl = 0.003531810361891985
 ----- On train/test for day 70 ----- 
On epoch 1 of day 70 training, loss = 0.00008622
On epoch 2 of day 70 training, loss = 0.00008202
On epoch 3 of day 70 training, loss = 0.00008007
On epoch 4 of day 70 training, loss = 0.00007921
On epoch 5 of day 70 training, loss = 0.00007782
On epoch 6 of day 70 training, loss = 0.00007660
On epoch 7 of day 70 training, loss = 0.00007544
On epoch 8 of day 70 training, loss = 0.00007443
On epoch 9 of day 70 training

On epoch 9 of day 76 training, loss = 0.00008749
On epoch 10 of day 76 training, loss = 0.00008680
On epoch 11 of day 76 training, loss = 0.00008601
On epoch 12 of day 76 training, loss = 0.00008555
On epoch 13 of day 76 training, loss = 0.00008538
On epoch 14 of day 76 training, loss = 0.00008513
On epoch 15 of day 76 training, loss = 0.00008472
On epoch 16 of day 76 training, loss = 0.00008428
On epoch 17 of day 76 training, loss = 0.00008395
On epoch 18 of day 76 training, loss = 0.00008371
On epoch 19 of day 76 training, loss = 0.00008345
On epoch 20 of day 76 training, loss = 0.00008316
On epoch 21 of day 76 training, loss = 0.00008286
On epoch 22 of day 76 training, loss = 0.00008261
On epoch 23 of day 76 training, loss = 0.00008239
On epoch 24 of day 76 training, loss = 0.00008216
On epoch 25 of day 76 training, loss = 0.00008194
Current test pnl = -0.0025127825792878866
 ----- On train/test for day 77 ----- 
On epoch 1 of day 77 training, loss = 0.00012034
On epoch 2 of day 77 

On epoch 1 of day 83 training, loss = 0.00011216
On epoch 2 of day 83 training, loss = 0.00011026
On epoch 3 of day 83 training, loss = 0.00010986
On epoch 4 of day 83 training, loss = 0.00010932
On epoch 5 of day 83 training, loss = 0.00010879
On epoch 6 of day 83 training, loss = 0.00010834
On epoch 7 of day 83 training, loss = 0.00010798
On epoch 8 of day 83 training, loss = 0.00010759
On epoch 9 of day 83 training, loss = 0.00010719
On epoch 10 of day 83 training, loss = 0.00010679
On epoch 11 of day 83 training, loss = 0.00010642
On epoch 12 of day 83 training, loss = 0.00010607
On epoch 13 of day 83 training, loss = 0.00010572
On epoch 14 of day 83 training, loss = 0.00010536
On epoch 15 of day 83 training, loss = 0.00010501
On epoch 16 of day 83 training, loss = 0.00010468
On epoch 17 of day 83 training, loss = 0.00010435
On epoch 18 of day 83 training, loss = 0.00010403
On epoch 19 of day 83 training, loss = 0.00010371
On epoch 20 of day 83 training, loss = 0.00010340
On epoch 

On epoch 9 of day 89 training, loss = 0.00005485
On epoch 10 of day 89 training, loss = 0.00005439
On epoch 11 of day 89 training, loss = 0.00005394
On epoch 12 of day 89 training, loss = 0.00005357
On epoch 13 of day 89 training, loss = 0.00005323
On epoch 14 of day 89 training, loss = 0.00005288
On epoch 15 of day 89 training, loss = 0.00005253
On epoch 16 of day 89 training, loss = 0.00005220
On epoch 17 of day 89 training, loss = 0.00005191
On epoch 18 of day 89 training, loss = 0.00005162
On epoch 19 of day 89 training, loss = 0.00005133
On epoch 20 of day 89 training, loss = 0.00005106
On epoch 21 of day 89 training, loss = 0.00005081
On epoch 22 of day 89 training, loss = 0.00005056
On epoch 23 of day 89 training, loss = 0.00005033
On epoch 24 of day 89 training, loss = 0.00005010
On epoch 25 of day 89 training, loss = 0.00004988
Current test pnl = 0.004433352034538984
 ----- On train/test for day 90 ----- 
On epoch 1 of day 90 training, loss = 0.00005764
On epoch 2 of day 90 tr

On epoch 14 of day 96 training, loss = 0.00002868
On epoch 15 of day 96 training, loss = 0.00002830
On epoch 16 of day 96 training, loss = 0.00002800
On epoch 17 of day 96 training, loss = 0.00002786
On epoch 18 of day 96 training, loss = 0.00002777
On epoch 19 of day 96 training, loss = 0.00002762
On epoch 20 of day 96 training, loss = 0.00002742
On epoch 21 of day 96 training, loss = 0.00002726
On epoch 22 of day 96 training, loss = 0.00002714
On epoch 23 of day 96 training, loss = 0.00002704
On epoch 24 of day 96 training, loss = 0.00002693
On epoch 25 of day 96 training, loss = 0.00002681
Current test pnl = -0.003002751851454377
 ----- On train/test for day 97 ----- 
On epoch 1 of day 97 training, loss = 0.00004780
On epoch 2 of day 97 training, loss = 0.00004361
On epoch 3 of day 97 training, loss = 0.00004297
On epoch 4 of day 97 training, loss = 0.00004258
On epoch 5 of day 97 training, loss = 0.00004240
On epoch 6 of day 97 training, loss = 0.00004206
On epoch 7 of day 97 train

# Saving results for analysis

In [49]:
save_string = f'{model_type}_{data.snapshot_count}days_{feature_size}feature_size_{GDN_epochs}GDNe_{DDPM_epochs}DDPMe_{gamma}gamma_{delta}delta_{GDN_lr}GDN_lr_{DDPM_lr}DDPM_lr_{DDPM_lb}DDPM_lb_{training_lb}_tr_lb___{data_name}'

with open(f'/{model_type}_outputs/{save_string}', 'wb') as f:
    pickle.dump(outputs_GDN, f)

torch.save(gdn.state_dict(), f'/{model_type}_weights/WEIGHTS_{save_string}')